In [1]:
import torch
import torch
import torch.nn as nn
from torch.nn import functional as F

c:\Users\Pc\miniconda3\envs\gait\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# with open("input.txt") as f:
with open("./cleaned_merged_fairy_tales_without_eos.txt") as f:
    text = f.read()



In [3]:
print("The length of total dataset: ",len(text))

The length of total dataset:  20455694


In [4]:
print("The first 100 characters of the dataset: \n",text[:100])

The first 100 characters of the dataset: 
 The Happy Prince.
HIGH above the city, on a tall column, stood the statue of the Happy Prince.  He w


In [5]:
char = sorted(set(text))

vocab_size = len(char)
print(char)
print(''.join(char))
print("The number of unique characters in the dataset: ",vocab_size)

['\n', ' ', '!', '"', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '\xa0', '£', '°', '½', 'Ä', 'Æ', 'É', 'à', 'â', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ñ', 'ô', 'ö', 'ü', 'ā', 'œ', '—', '‘', '’', '“', '”', '…']

 !"$&'()*+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]`abcdefghijklmnopqrstuvwxyz{|} £°½ÄÆÉàâäæçèéêëîïñôöüāœ—‘’“”…
The number of unique characters in the dataset:  117


In [6]:
stoi = {chi:i for i,chi in enumerate(char) }
itos = {i:chi for i,chi in enumerate(char) }

encoder = lambda x: [stoi[chari] for chari in x]
decoder = lambda x: ''.join([itos[chari] for chari in x])


print(encoder("hii there"))
print(decoder([46, 47, 47, 1, 58, 46, 43, 56, 43]))

print(''.join(['cd', 'e']))

[65, 66, 66, 1, 77, 65, 62, 75, 62]
RSS aRO]O
cde


In [7]:

data = torch.tensor(encoder(text))

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
batch_size = 256 # how many independent sequences will we process in parallel?   # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions? # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embedding = 256
head_size = 4
n_layer = 4
dropout = 0.0


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"The context is: {context} and the target is: {target}")




torch.manual_seed(1337)

def get_batch(split, device = 'cpu'):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

The context is: tensor([48]) and the target is: 65
The context is: tensor([48, 65]) and the target is: 62
The context is: tensor([48, 65, 62]) and the target is: 1
The context is: tensor([48, 65, 62,  1]) and the target is: 36
The context is: tensor([48, 65, 62,  1, 36]) and the target is: 58
The context is: tensor([48, 65, 62,  1, 36, 58]) and the target is: 73
The context is: tensor([48, 65, 62,  1, 36, 58, 73]) and the target is: 73
The context is: tensor([48, 65, 62,  1, 36, 58, 73, 73]) and the target is: 82
The context is: tensor([48, 65, 62,  1, 36, 58, 73, 73, 82]) and the target is: 1
The context is: tensor([48, 65, 62,  1, 36, 58, 73, 73, 82,  1]) and the target is: 44
The context is: tensor([48, 65, 62,  1, 36, 58, 73, 73, 82,  1, 44]) and the target is: 75
The context is: tensor([48, 65, 62,  1, 36, 58, 73, 73, 82,  1, 44, 75]) and the target is: 66
The context is: tensor([48, 65, 62,  1, 36, 58, 73, 73, 82,  1, 44, 75, 66]) and the target is: 71
The context is: tensor([48,

In [9]:
@torch.no_grad()
def estimateLoss(model, device = 'cpu'):
    out = {}
    # print(device)
    model = model.to(device)
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split, device)
            # print(x.device, y.device, model)
            x = x.to(device)
            y = y.to(device)
            model = model.to(device)
            logits, loss = model(x , y)
            losses[k] = loss.item()
        out[split] = losses.mean()
        model.train()
    return out

In [10]:
batch_size = 256 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embedding = 256
head_size = 4
n_layer = 4
dropout = 0.0

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embedding, head_size, bias=False)
        self.query = nn.Linear(n_embedding, head_size, bias=False)
        self.value = nn.Linear(n_embedding, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out
    

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.head = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embedding, n_embedding)
    def forward(self,x):
        # Concatenare over the channel dimension
        out = torch.cat([h(x) for h in self.head], dim=-1)
        out = self.proj(out)

        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    


class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel_simple(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embedding) # n_embedding to make the model a little longer
        self.position_embedding = nn.Embedding(block_size, n_embedding) # n_embedding to make the model a little longer
        # self.head = Head(n_embedding)
        self.m_head = MultiHeadAttention(4 , n_embedding//4)
        self.lm_head = nn.Linear(n_embedding, vocab_size)
        self.feed_forward = FeedFoward(n_embedding)
    def forward(self, idx, targets=None):
        B,T = idx.shape
        # print("Input: ", idx.shape)
        # idx and targets are both (B,T) tensor of integers
        tok_embedd = self.token_embedding_table(idx) # (B,T,C)
        # print("Token Embedding: ",tok_embedd.shape)
        pos_embedd =  self.position_embedding(torch.arange(T))  # T,C
        # print("Positional Embedding: ",pos_embedd.shape)
        x = pos_embedd + tok_embedd # B,T,C
        # print("Added Embedding: ",x.shape)
        x = self.m_head(x) # apply one head of the self-attention (B,T,C)
        x = self.feed_forward(x)
        # print("After single head: ",x.shape)
        logits = self.lm_head(x)  # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size
            # We cannot enter the embedding greater than batch_size
            # print("----------idx------------",idx.shape)
            idx_cond = idx[:, -block_size:]
            # print("----------idx_cond------------",idx_cond.shape)
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution. only num_sample prediction from the logits 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            # print(idx.shape, idx_next.shape)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embedding) # n_embedding to make the model a little longer
        self.position_embedding = nn.Embedding(block_size, n_embedding) # n_embedding to make the model a little longer
        # self.head = Head(n_embedding)
        self.blocks = nn.Sequential(
            Block(n_embedding, n_head=4 ),
            Block(n_embedding, n_head=4 ),
            Block(n_embedding, n_head=4 ),
        )
        self.l_norm = nn.LayerNorm(n_embedding)
        self.lm_head = nn.Linear(n_embedding, vocab_size)
    def forward(self, idx, targets=None):
        B,T = idx.shape
        # print("Input: ", idx.shape)
        # idx and targets are both (B,T) tensor of integers
        tok_embedd = self.token_embedding_table(idx) # (B,T,C)
        # print("Token Embedding: ",tok_embedd.shape)
        pos_embedd =  self.position_embedding(torch.arange(T, device = device))  # T,C
        # print("Positional Embedding: ",pos_embedd.shape)
        x = pos_embedd + tok_embedd # B,T,C
        # print("Added Embedding: ",x.shape)
        x = self.blocks(x) # apply one head of the self-attention (B,T,C)
        # print("After single head: ",x.shape)
        x = self.l_norm(x)

        logits = self.lm_head(x)  # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size
            # We cannot enter the embedding greater than batch_size
            # print("----------idx------------",idx.shape)
            idx_cond = idx[:, -block_size:]
            # print("----------idx_cond------------",idx_cond.shape)
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution. only num_sample prediction from the logits 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            # print(idx.shape, idx_next.shape)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
# logits, loss = model(xb, yb)
# print(logits.shape)
# print(loss)
model = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

2.435701 M parameters


In [11]:


# batch_size = 32
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for  iter in range(max_iters):
    if iter % 100 == 0:
        losses = estimateLoss(model, device = device)
        print(f"iter {iter}, train loss: {losses['train']}, val loss: {losses['val']}")

    xb, yb = get_batch('train', device)
    # xb = xb.to(device)
    # yb = yb.to(device)
    lotics, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    torch.save(model.state_dict(), "model.pt")
        # print(decoder(m.generate(idx = xb[:1], max_new_tokens=100)[0].tolist()))


print(f"The loss is: {loss.item()}")






iter 0, train loss: 4.9347028732299805, val loss: 4.933953762054443
iter 100, train loss: 2.41334867477417, val loss: 2.415299892425537
iter 200, train loss: 2.1522724628448486, val loss: 2.1906421184539795
iter 300, train loss: 2.0111234188079834, val loss: 2.0575191974639893
iter 400, train loss: 1.9251960515975952, val loss: 1.9851429462432861
iter 500, train loss: 1.8684930801391602, val loss: 1.9292552471160889
iter 600, train loss: 1.8278111219406128, val loss: 1.8907558917999268
iter 700, train loss: 1.7951812744140625, val loss: 1.8565319776535034
iter 800, train loss: 1.7661728858947754, val loss: 1.830472707748413
iter 900, train loss: 1.7407749891281128, val loss: 1.8070385456085205
iter 1000, train loss: 1.715061902999878, val loss: 1.7875487804412842
iter 1100, train loss: 1.6971174478530884, val loss: 1.76694917678833
iter 1200, train loss: 1.6836007833480835, val loss: 1.760208010673523
iter 1300, train loss: 1.6636117696762085, val loss: 1.7386415004730225
iter 1400, tr

In [36]:
model.load_state_dict(torch.load("./model.pt"))
# model.eval()

<All keys matched successfully>

In [37]:
def get_answer():
    input_data = input("Please enter the prompt: ")
    encoded_data = encoder(input_data)
    print(encoded_data)
    encoded_data = torch.tensor(encoded_data)
    encoded_data = encoded_data[None, :]
    encoded_data = encoded_data.to(device)
    print(encoded_data.shape)
    out = model.generate(idx = torch.tensor(encoded_data[:]), max_new_tokens=100)[0].detach().tolist()
    out = decoder(out)
    print(out)

get_answer()

[65, 62, 69, 69, 72]
torch.Size([1, 5])


C:\Users\Pc\AppData\Local\Temp\ipykernel_328\4286314306.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = model.generate(idx = torch.tensor(encoded_data[:]), max_new_tokens=100)[0].detach().tolist()


hellow's brail, but, a man, bit turn-which you anything anefusarite several chil, for I don't wish a quot
